In [30]:
import tweepy
from twitter_authentication import bearer_token
import time
import pandas as pd
import json
import pickle

In [31]:
def json_load(filepath):
    with open(filepath, "r") as json_file:
        data = json.load(json_file)
    return data

def json_dump(data, filepath, pretty_format = True):
    with open(filepath, 'w') as fw:
        if pretty_format:
            json.dump(data, fw, indent=2, sort_keys=True)
        else:
            json.dump(data, fw)

def pickle_dump(obj, pickle_filepath):
    with open(pickle_filepath, "wb") as f:
        pickle.dump(obj, f, protocol=2)

def pickle_load(pickle_filepath):
    with open(pickle_filepath, "rb") as f:
        obj = pickle.load(f)
    return obj

In [32]:
client = tweepy.Client(bearer_token, wait_on_rate_limit=True)

In [ ]:
next_token = None
place_dict = {}
user_dict = {}
tweet_dict = {}



place_dict, user_dict, tweet_dict, next_token = get_tweets(place_dict,user_dict,tweet_dict, next_token)

In [58]:
while next_token is not None:
    place_dict, user_dict, tweet_dict, next_token = get_tweets(place_dict,user_dict,tweet_dict, next_token)

In [62]:
def get_tweets(place_dict = {},
               user_dict = {},
               tweet_dict = {},
                next_token = None,
               query = 'alzheimers disease OR alzheimer -is:retweet place_country:US',
              user_fields = ['username', 'public_metrics', 'description', 'location'],
              place_fields = ['place_type', 'geo'],
              tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
              expansions = ['author_id', 'geo.place_id'],
              start_time = '2018-01-01T00:00:00Z',
               end_time = '2020-01-15T00:00:00Z'
              ):
    place_keys= ["full_name", 'id', 'contained_within', 'country', 'country_code', 'geo', 'name', 'place_type' ]
    user_keys= ['id', 'name', 'username', 'created_at', 
            'description', "entities", 'location', 'pinned_tweet_id', 
            'profile_image_url', 'protected', 'public_metrics', 'url', 'verified', 'withheld']
    tweet_keys = ['id', 'text', 'author_id', 'context_annotations',
             'conversation_id', 'entities', 'in_reply_to_user_id', 'lang',
             'non_public_metrics', 'organic_metrics', 'possibly_sensitive',
             'promoted_metrics', 'public_metrics', 'referenced_tweets', 'reply_settings',
             'source', 'withheld']    


    for tweet in tweepy.Paginator(client.search_all_tweets, 
                                     query = query,
                                     user_fields = user_fields,
                                     place_fields = place_fields,
                                     tweet_fields = tweet_fields,
                                     expansions = expansions,
                                     start_time = start_time,
                                     end_time = end_time,
                                  pagination_token = next_token,
                                  max_results=500):
        time.sleep(1)
        alz_tweets.append(tweet)
    
    
    # Loop through each response object
    for response in alz_tweets:
        # Take all of the users, and put them into a dictionary of dictionaries with the info we want to keep
        for place in response.includes['places']:
            place_obj = {}
            for key in place_keys:
                place_obj[key] = place[key]
            place_dict[place_obj['id']] = place_obj
        for user in response.includes['users']:
            user_obj = {}
            for key in user_keys:
                user_obj[key] = user[key]
            user_dict[user_obj['id']] = user_obj

        for tweet in response.data:
            tweet_obj = {}
            for key in tweet_keys:
                tweet_obj[key] = getattr(tweet, key)
            tweet_dict[tweet_obj['id']] = tweet_obj
#     if len(alz_tweets) == 0:
#         next_token = None
#     else:
#         if 'next_token' is in alz_tweets[0].meta:
#             next_token = alz_tweets[0].meta['next_token']
    
    return place_dict, user_dict, tweet_dict, next_token

    print(len(alz_tweets))


In [60]:
alz_tweets[0].meta['next_token'], alz_tweets[1].meta['next_token']

('b26v89c19zqg8o3fnlo0yxc7761hth3gu5oeu7ti2ma2l',
 'b26v89c19zqg8o3fn0dor7lfae0eh6204lplge3rqkv3x')

In [61]:
for idx, response in enumerate(alz_tweets):
    print(idx, len(response.data))

0 497
1 491
2 342


In [115]:
print(len(alz_tweets))

3


In [116]:
alz_tweets[-1].meta

{'newest_id': '1017433152736804870',
 'oldest_id': '947865213335998464',
 'result_count': 339}

In [23]:
response = alz_tweets[0]
response.includes.keys()

dict_keys(['users', 'places'])

In [69]:
json_dump(place_dict, filepath = "place.json")

In [70]:
json_dump(user_dict, filepath = "user.json")

In [71]:
json_dump(tweet_dict, filepath = "tweet.json")

In [12]:
#Place Attributes
place_keys= ["full_name", 'id', 'contained_within', 'country', 'country_code', 'geo', 'name', 'place_type' ]
place_dict = {}
#User Attributes
user_keys= ['id', 'name', 'username', 'created_at', 
            'description', "entities", 'location', 'pinned_tweet_id', 
            'profile_image_url', 'protected', 'public_metrics', 'url', 'verified', 'withheld']
user_dict = {}
#Tweet Attributes
tweet_keys = ['id', 'text', 'author_id', 'context_annotations',
             'conversation_id', 'entities', 'in_reply_to_user_id', 'lang',
             'non_public_metrics', 'organic_metrics', 'possibly_sensitive',
             'promoted_metrics', 'public_metrics', 'referenced_tweets', 'reply_settings',
             'source', 'withheld']
tweet_dict = {}
full_list=[]

for response in alz_tweets: 
    for place in response.includes['places']:
        place_obj = {}
        for key in place_keys:
            place_obj[key] = place[key]
        place_dict[place_obj['id']] = place_obj
    for user in response.includes['users']:
        user_obj = {}
        for key in user_keys:
            user_obj[key] = user[key]
        user_dict[user_obj['id']] = user_obj
    for tweet in response.data:
        tweet_obj = {}
        for key in tweet_keys:
            tweet_obj[key] = getattr(tweet, key)
        tweet_dict[tweet_obj['id']] = tweet_obj
    full_list.append(place_obj)
    full_list.append(user_obj)
    full_list.append(tweet_obj)
    
        
    

In [13]:
full_list

[{'full_name': 'San Marcos, CA',
  'id': 'a2c84129f9dcf69f',
  'contained_within': [],
  'country': None,
  'country_code': None,
  'geo': {'type': 'Feature',
   'bbox': [-117.2301723, 33.0907611, -117.103461, 33.186722],
   'properties': {}},
  'name': None,
  'place_type': 'city'},
 {'id': 857761126305730560,
  'name': 'Amanda Brandeis',
  'username': 'amanda_brandeis',
  'created_at': None,
  'description': 'Visual Storyteller + Emmy Award Winning Journalist',
  'entities': None,
  'location': 'San Diego, CA',
  'pinned_tweet_id': None,
  'profile_image_url': None,
  'protected': None,
  'public_metrics': {'followers_count': 1082,
   'following_count': 530,
   'tweet_count': 1005,
   'listed_count': 21},
  'url': None,
  'verified': None,
  'withheld': None},
 {'id': 981221671792123904,
  'text': "A hiker has officially set off from San Diego on a 2,650-mile hike. \n\nHe's raising money for a disease that robs millions of their memories. More info on how to follow his journey or don